In [ ]:
from transformers import AutoTokenizer, BitsAndBytesConfig, pipeline, AutoModelForCausalLM
import torch
import json
from pathlib import Path
import numpy as np
import pandas as pd

In [ ]:
from huggingface_hub import login
hf_api_key = ""
login(hf_api_key)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import os
os.chdir("")

In [ ]:
file_path = 'formulas_with_explanations_with_questions_with_gt.xlsx'
df_all = pd.read_excel(file_path)

In [ ]:
df = df_all[:1]

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)
model = AutoModelForCausalLM.from_pretrained("meta-llama/Meta-Llama-3-8B-Instruct").to(device)
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B-Instruct")
print("load done!")

cuda


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


load done!


In [ ]:
def evaluate_llm(questions, answers, formula=None):
    results = []
    max_length = model.config.max_position_embeddings if hasattr(model.config, 'max_position_embeddings') else 2048  # Default to 2048 if not specified

    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    for i in range(len(questions)):
        if not formula:
            prompt = f"""
                  You are a financial calculation expert. Please answer the following question and only return the final calculation result.

                  Input question: 'A project expects annual cash inflows of $6,000 for 4 years. If the discount rate is 8%, what is the NPV of the project?'
                  Output: '21462.58'

                  Input question: 'Starting Investment (PV): $2,000\nAnnual Interest Rate (r): 3%\nInvestment Duration (n): 10 years\nWhat will be the Future Value (FV) of this investment?'
                  Output: '2687.83'

                  Input question: 'Suppose you anticipate receiving $10,000 in 5 years. If the annual discount rate is 7%, what is the present value of this sum?'
                  Output: '7129.86'

                  Input questions: 'Principal Amount (P): $10,000, Annual Interest Rate (r): 4%, Compounded Quarterly (n=4), Duration of Investment (t): 5 years. Calculate the future value using the compound interest formula.'
                  Output: '12201.9'

                  Input questions: 'You invest $2,000 at an annual interest rate of 4%. How much interest will you earn after 5 years?'
                  Output: '400'

                  Input question: {questions[i]}
                  So the output is
                  """
        else:
            prompt = f"""
                  You are a financial calculation expert. Please answer the following question based on the following formula and return only the final calculation result.

                  Input formula: 'Net Present Value (NPV), NPV = ∑ (R_t / (1 + i)^t), R_t: Net cash inflow during the period t, i: Discount rate, t: Number of time periods'
                  Input question: 'A project expects annual cash inflows of $6,000 for 4 years. If the discount rate is 8%, what is the NPV of the project?'
                  Output: '21462.58'

                  Input formula: 'Future Value (FV), FV = PV × (1 + r)^n, PV: Present value, r: Interest rate, n: Number of periods'
                  Input question: 'Starting Investment (PV): $2,000\nAnnual Interest Rate (r): 3%\nInvestment Duration (n): 10 years\nWhat will be the Future Value (FV) of this investment?'
                  Output: '2687.83'

                  Input formula: 'Present Value (PV), PV = FV / (1 + r)^n, FV: Future value, r: Interest rate, n: Number of periods'
                  Input question: 'Suppose you anticipate receiving $10,000 in 5 years. If the annual discount rate is 7%, what is the present value of this sum?'
                  Output: '7129.86'

                  Input formula: 'Compound Interest, A = P (1 + r/n)^(nt), A: Amount of money accumulated, P: Principal amount, r: Annual interest rate, n: Number of times interest is compounded per year, t: Time the money is invested for'
                  Input questions: 'Principal Amount (P): $10,000, Annual Interest Rate (r): 4%, Compounded Quarterly (n=4), Duration of Investment (t): 5 years. Calculate the future value using the compound interest formula.'
                  Output: '12201.9'

                  Input formula: 'Simple Interest, I = P × r × t, I: Interest, P: Principal amount, r: Annual interest rate, t: Time the money is invested for'
                  Input questions: 'You invest $2,000 at an annual interest rate of 4%. How much interest will you earn after 5 years?'
                  Output: '400'

                  Input formula: {formula[i]}
                  Input question: {questions[i]}
                  So the output is
                  """

        messages = [
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ]
        text = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
        )

        inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=max_length)
        input_ids = inputs.input_ids.to(device)
        attention_mask = inputs.attention_mask.to(device)

        remaining_tokens = max_length - input_ids.shape[1]
        max_new_tokens = min(512, remaining_tokens)

        generated_ids = model.generate(
            input_ids,
            attention_mask=attention_mask,
            max_new_tokens=max_new_tokens,
            pad_token_id=tokenizer.pad_token_id
        )

        generated_ids = generated_ids[:, input_ids.shape[1]:]
        generated_answer = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
        results.append((questions[i], generated_answer, answers[i]))

    return results

In [ ]:
results_df_without_formula = pd.DataFrame(columns=["Formula", "Question", "Generated Answer", "Correct Answer"])
results_df_with_formula = pd.DataFrame(columns=["Formula", "Question", "Generated Answer", "Correct Answer"])

file_without_formula = "formula_generated_results_without_formula_llama3_8b.xlsx"
file_with_formula = "formula_generated_results_with_formula_llama3_8b.xlsx"

results_df_without_formula.to_excel(file_without_formula, index=False)
results_df_with_formula.to_excel(file_with_formula, index=False)


In [ ]:
for j in range(len(df)):
    questions = df[[f"Question {i}" for i in range(1, 21)]].values[j]
    answers = df[[f"Answer {i}" for i in range(1, 21)]].values[j]
    formulas = [df["Formula Name"].values[j] + " " + df["Formula"].values[j] + " " + df["Formula"].values[j]] * 20

    results_wf = evaluate_llm(questions, answers)
    for result in results_wf:
        new_row = pd.DataFrame({
            "Formula": [df["Formula Name"].values[j]],
            "Question": [result[0]],
            "Generated Answer": [result[1]],
            "Correct Answer": [result[2]],
        })

        with pd.ExcelWriter(file_without_formula, engine='openpyxl', mode='a', if_sheet_exists='overlay') as writer:
            startrow = writer.sheets['Sheet1'].max_row
            new_row.to_excel(writer, index=False, header=False, startrow=startrow)


    results_wf = evaluate_llm(questions, answers, formulas)
    for result in results_wf:
        new_row = pd.DataFrame({
            "Formula": [df["Formula Name"].values[j]],
            "Question": [result[0]],
            "Generated Answer": [result[1]],
            "Correct Answer": [result[2]],
        })

        with pd.ExcelWriter(file_with_formula, engine='openpyxl', mode='a', if_sheet_exists='overlay') as writer:
            startrow = writer.sheets['Sheet1'].max_row
            new_row.to_excel(writer, index=False, header=False, startrow=startrow)

    print(f"Processed and saved results for formula {j+1}/{len(df)}")

print("Finished processing all formulas")